In [ ]:

!pip uninstall -y torch torchvision torchaudio diffusers transformers accelerate xformers bitsandbytes peft datasets huggingface_hub safetensors


!pip install diffusers==0.30.0 transformers accelerate safetensors torchvision torch --upgrade
!pip install -q diffusers==0.30.0 transformers accelerate safetensors bitsandbytes datasets xformers peft
!pip -q install huggingface_hub datasets


!pip install git+https://github.com/openai/CLIP.git


import diffusers
import numpy as np
import random


import torch
from huggingface_hub import snapshot_download
from datasets import load_dataset, Image, Value, Features
import os


repo_dir = snapshot_download("ffurfaro/PixelBytes-PokemonAll", repo_type="dataset")
pq = os.path.join(repo_dir, "data", "train-00000-of-00001.parquet")
ds = load_dataset("parquet", data_files=pq)["train"]
ds = ds.remove_columns([c for c in ds.column_names if c not in ["image", "text"]])


print(ds.column_names, len(ds))


def load_model():


 MODEL_ID = "OFA-Sys/small-stable-diffusion-v0"
 device = "cuda" if torch.cuda.is_available() else "cpu"


 # load the pipeline (FP16 if GPU supports it)
 pipe = diffusers.StableDiffusionPipeline.from_pretrained(MODEL_ID, torch_dtype=torch.float16 if device=="cuda" else torch.float32)
 pipe = pipe.to(device)
 # pipe.enable_xformers_memory_efficient_attention() if hasattr(pipe, "enable_xformers_memory_efficient_attention") else None


 try:
     pipe.enable_xformers_memory_efficient_attention()
 except Exception as e:
     print("⚠️ Skipping xformers:", e)


 print("Loaded model:", MODEL_ID, "on", device)




 return pipe, device


pipe, device = load_model()


save_dir = "/content/pokemon/train"
os.makedirs(save_dir, exist_ok=True)


for i, sample in enumerate(ds):
   # Save image
   img = sample["image"]      # PIL.Image
   img.save(f"{save_dir}/{i:04d}.png")


   # Save caption text
   caption = sample["text"]
   with open(f"{save_dir}/{i:04d}.txt", "w") as f:
       f.write(caption)


print("Done! Saved", len(ds), "images + captions to:", save_dir)


# --- Full Rank Fine Tuning ---


def full_rank(pipe, device):
 unet = pipe.unet




 # Make every parameter in the UNet trainable
 for p in unet.parameters():
     p.requires_grad = True


 unet.to(device)
 pipe.unet = unet


 # Get all trainable parameters (this will now be the full UNet)
 trainable = [p for p in unet.parameters() if p.requires_grad]
 print("✅ Full Rank. Trainable params:", sum(p.numel() for p in trainable))


 return unet, trainable


unet, trainable = full_rank(pipe, device)


import matplotlib.pyplot as plt
import torch
import clip


# --------------------------------------
# Storage for logs
# --------------------------------------
train_losses = []
val_losses = []
clip_scores = []


# --------------------------------------
# CLIP model for similarity
# --------------------------------------
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)


from torchvision.transforms.functional import to_pil_image


def compute_clip_score(imgs, texts):
   """
   imgs: tensor batch in [-1,1]
   texts: list of captions
   """


   # ---- 1. Truncate long captions for CLIP (max context length 77 tokens) ----
   # Hard truncate to ~200 chars (safe for CLIP)
   texts = [t[:200] for t in texts]


   # ---- 2. Convert tensors → PIL ----
   pil_imgs = []
   for img in imgs:
       img = (img * 0.5 + 0.5).clamp(0,1)
       pil_imgs.append(to_pil_image(img.cpu()))


   # ---- 3. Preprocess for CLIP ----
   imgs_pre = torch.stack([clip_preprocess(p) for p in pil_imgs]).to(device)


   # truncate=True prevents CLIP errors
   text_tokens = clip.tokenize(texts, truncate=True).to(device)


   with torch.no_grad():
       img_feat = clip_model.encode_image(imgs_pre)
       txt_feat = clip_model.encode_text(text_tokens)


   img_feat /= img_feat.norm(dim=-1, keepdim=True)
   txt_feat /= txt_feat.norm(dim=-1, keepdim=True)


   similarity = (img_feat * txt_feat).sum(dim=-1)
   return similarity.mean().item()




# --------------------------------------
# Validation step
# --------------------------------------
def validate(val_loader, vae, text_encoder, tokenizer, noise_scheduler):
   unet.eval()
   total_loss = 0
   count = 0


   with torch.no_grad():
       for imgs, captions in val_loader:
           imgs = imgs.to(device, dtype=torch.float16 if device=="cuda" else torch.float32)


           latents = vae.encode(imgs).latent_dist.sample() * 0.18215


           # Add noise
           noise = torch.randn_like(latents)
           timesteps = torch.randint(
               0, noise_scheduler.config.num_train_timesteps,
               (latents.shape[0],), device=device
           ).long()
           noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)


           # Text encode
           text_inputs = tokenizer(
               captions, padding="max_length", truncation=True,
               max_length=tokenizer.model_max_length,
               return_tensors="pt"
           ).to(device)


           enc = text_encoder(text_inputs.input_ids)[0]


           # Predict noise
           pred = unet(noisy_latents, timesteps, encoder_hidden_states=enc).sample
           loss = ((pred - noise) ** 2).mean()


           total_loss += loss.item()
           count += 1


   unet.train()
   return total_loss / count




# --------------------------------------
# Plotting function
# --------------------------------------
def plot_losses(train_losses, val_losses, learning_rate, weight_decay):
   epochs = range(1, len(train_losses) + 1)
   plt.figure(figsize=(8,5))
   plt.plot(epochs, train_losses, label="Train Loss")
   plt.plot(epochs, val_losses, label="Validation Loss")
   plt.xlabel("Epoch")
   plt.ylabel("Loss")
   plt.title(f"Train vs Validation Loss (lr={learning_rate}, wd={weight_decay})")
   plt.legend()
   plt.grid(True)
   plt.show()




def plot_clip_scores(clip_scores, learning_rate, weight_decay):
   epochs = range(1, len(clip_scores) + 1)
   plt.figure(figsize=(8,5))
   plt.plot(epochs, clip_scores, label="CLIP Score")
   plt.xlabel("Epoch")
   plt.ylabel("CLIP Score")
   plt.title(f"CLIP Score per Epoch (lr={learning_rate}, wd={weight_decay})")
   plt.grid(True)
   plt.show()


from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm.auto import tqdm
import torch
from torch.optim import AdamW
import os


class SimpleDataset(Dataset):
   def __init__(self, folder):
       self.files = sorted([f for f in os.listdir(folder) if f.endswith(".png")])
       self.folder = folder
       self.transform = transforms.Compose([
           transforms.Resize((512,512), interpolation=Image.NEAREST),
           transforms.ToTensor(),
           transforms.Normalize([0.5]*3, [0.5]*3)
       ])
   def __len__(self):
       return len(self.files)
   def __getitem__(self, idx):
       img_path = os.path.join(self.folder, self.files[idx])
       txt_path = img_path.replace(".png", ".txt")


       img = Image.open(img_path).convert("RGB")
       with open(txt_path, "r") as f:
           caption = f.read().strip()
       return self.transform(img), caption






def train(learning_rate, weight_decay, num_epochs=10, batch_size=4): # Change batch size to 4


   pipe, device = load_model()
   unet, trainable = full_rank(pipe, device)






   dataset = SimpleDataset(save_dir)
   loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


   val_size = max(1, len(dataset) // 10)
   train_subset, val_subset = torch.utils.data.random_split(dataset, [len(dataset)-val_size, val_size])
   train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
   val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)


   optimizer = AdamW(trainable, lr=learning_rate, weight_decay=weight_decay)


   vae = pipe.vae
   noise_scheduler = pipe.scheduler
   tokenizer = pipe.tokenizer
   text_encoder = pipe.text_encoder
   unet.train()


   # Clear logs for new run
   train_losses.clear()
   val_losses.clear()
   clip_scores.clear()


   for epoch in range(num_epochs):


       epoch_loss = 0.0
       batches = 0


       for imgs, captions in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
           imgs = imgs.to(device, dtype=torch.float16 if device=="cuda" else torch.float32)


           with torch.no_grad():
               latents = vae.encode(imgs).latent_dist.sample() * 0.18215


           noise = torch.randn_like(latents)
           timesteps = torch.randint(
               0, noise_scheduler.config.num_train_timesteps,
               (latents.shape[0],), device=device
           ).long()
           noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)


           text_inputs = tokenizer(
               captions,
               padding="max_length",
               max_length=tokenizer.model_max_length,
               truncation=True,
               return_tensors="pt"
           ).to(device)


           with torch.no_grad():
               encoder_hidden_states = text_encoder(text_inputs.input_ids)[0]


           pred = unet(noisy_latents, timesteps, encoder_hidden_states=encoder_hidden_states).sample
           loss = ((pred - noise) ** 2).mean()
           # print(loss)


           optimizer.zero_grad()
           loss.backward()
           optimizer.step()


           epoch_loss += loss.item()
           batches += 1


       avg_train_loss = epoch_loss / batches
       train_losses.append(avg_train_loss)


       # ----- VALIDATION -----
       val_loss = validate(val_loader, vae, text_encoder, tokenizer, noise_scheduler)
       val_losses.append(val_loss)


       # ----- CLIP SCORE -----
       clip_score = compute_clip_score(*next(iter(val_loader)))
       clip_scores.append(clip_score)


       print(f"Epoch {epoch+1}: train={avg_train_loss:.4f}, val={val_loss:.4f}, clip={clip_score:.4f}")


       unet.save_pretrained(f"/content/adamw/lora_epoch_{epoch+1}")




   print("Training complete.")


   # ---------------------------------------------------------
   #                   PLOT RESULTS
   # ---------------------------------------------------------
   plot_losses(train_losses, val_losses, learning_rate, weight_decay)
   # plot_clip_scores(clip_scores, learning_rate, weight_decay)


   # avg_train = sum(train_losses) / len(train_losses)
   # avg_val   = sum(val_losses) / len(val_losses)
   # avg_clip  = sum(clip_scores) / len(clip_scores)


   print("\n===== FINAL METRICS =====")
   print(f"Training Loss:    {train_losses[-1]:.4f}")
   print(f"Validation Loss:  {val_losses[-1]:.4f}")
   print(f"CLIP Score:       {clip_scores[-1]:.4f}")


   return {
       'avg_train_loss': train_losses[-1],
       'avg_val_loss': val_losses[-1],
       'avg_clip_score': clip_scores[-1],
       'all_train_loss' : train_losses[:-1],
       'all_val_loss' : val_losses[:-1],
       'all_clip_score' : clip_scores[:-1]
   }


lrs = [1e-6, 1e-7, 1e-8, 1e-9] # MODEL DIVERGES FOR LR > 1e-6
wds = [1e-1, 1e-2, 1e-3, 1e-4]
INDIVIDUAL_RUN = False
EPOCHS = 10




if INDIVIDUAL_RUN:
 lrs = [1e-6]
 wds = [1e-2]


result = {}
#hyperparameter tuning
for lr in lrs:
 for wd in wds:
   print(f"RUNNINNG FOR:")
   print(f"Learning Rate: {lr}")
   print(f"Weight Decay: {wd}")
   result[(lr, wd)] = train(
       learning_rate=lr,
       weight_decay=wd,
       num_epochs=EPOCHS
   )
   print(result)
   print("-"*50)


pipe.safety_checker = lambda images, clip_input: (images, [False] * images.shape[0])


prompt = "a cute pixel art dragon pokemon creature"
image = pipe(prompt, num_inference_steps=20).images[0]
image.save("/content/adamw/test_output.png")
print("✅ Saved:", "/content/adamw/test_output.png")


import numpy as np
import matplotlib.pyplot as plt


def run_multiple(learning_rate, weight_decay, num_epochs=10, batch_size=4, runs=5):
   """
   Runs your training function N times and computes error bars.
   """


   all_train = []
   all_val = []


   for r in range(runs):
       print(f"\n============================")
       print(f"🔥 RUN {r+1}/{runs}")
       print(f"============================")


       out = train(
           learning_rate=learning_rate,
           weight_decay=weight_decay,
           num_epochs=num_epochs,
           batch_size=batch_size,
       )


       # Copy the global logs after this run
       all_train.append(train_losses.copy())
       all_val.append(val_losses.copy())


   # Convert to NumPy arrays → shape (runs, epochs+1)
   all_train = np.array(all_train)
   all_val = np.array(all_val)


   # Compute mean/stdev along axis=0
   train_mean = all_train.mean(axis=0)
   train_std  = all_train.std(axis=0)


   val_mean = all_val.mean(axis=0)
   val_std  = all_val.std(axis=0)


   # -----------------------------
   #    Plot with error bars
   # -----------------------------
   epochs = np.arange(len(train_mean))


   plt.figure(figsize=(10,6))


   plt.errorbar(
       epochs, train_mean, yerr=train_std,
       fmt='-o', capsize=4, label="Train Loss (mean ± std)"
   )
   plt.errorbar(
       epochs, val_mean, yerr=val_std,
       fmt='-o', capsize=4, label="Val Loss (mean ± std)"
   )


   plt.ylim(0.10, 0.50)
   plt.xlabel("Epoch")
   plt.ylabel("Loss")
   plt.title(f"Loss w/ Error Bars over {runs} Runs")
   plt.grid(True)
   plt.legend()
   plt.show()


   # -----------------------------------
   #  Print epoch-by-epoch summary
   # -----------------------------------
   print("\n===== MEAN ± STD PER EPOCH =====")
   for e in range(len(train_mean)):
       print(f"Epoch {e}: "
             f"Train {train_mean[e]:.4f} ± {train_std[e]:.4f} | "
             f"Val {val_mean[e]:.4f} ± {val_std[e]:.4f}")


   return {
       "train_mean": train_mean,
       "train_std":  train_std,
       "val_mean":   val_mean,
       "val_std":    val_std,
       "raw_train":  all_train,
       "raw_val":    all_val,
   }


results = run_multiple(
   learning_rate=1e-6,
   weight_decay=0.1,
   num_epochs=10,
   batch_size=4,
   runs=5
)
